In [1]:
from log_parser import parse_keyboard_log
from log_parser import extract_predefined_patterns
from log_parser import parse_mouse_log
from keyboard_heatmap import KeyboardHeatmap
dataframe = parse_keyboard_log("logs/ZIR/SOL/key.log")
keyboard_df = extract_predefined_patterns(dataframe, 0, 10000)
print(keyboard_df)

  key  avg_duration  longestDuration  shortestDuration   class
0  sw      0.180105           0.2962            0.0791  ZIRSOL
1  ws      0.185755           0.2999            0.0841  ZIRSOL


In [2]:
dataframe = parse_keyboard_log("logs/HEN/SOL/key.log")
keyboard_df = extract_predefined_patterns(dataframe, 0, 10000)
print(keyboard_df)

  key  avg_duration  longestDuration  shortestDuration   class
0  sw      0.150597           0.2927            0.0107  HENSOL
1  ws      0.137060           0.2952            0.0533  HENSOL
2  w2      0.125550           0.2164            0.0738  HENSOL
3  2w      0.120033           0.2236            0.0558  HENSOL


In [3]:
dataframe = parse_keyboard_log("logs/ZIR/SNI/key.log")
keyboard_df = extract_predefined_patterns(dataframe, 0, 10000)
print(keyboard_df)

  key  avg_duration  longestDuration  shortestDuration   class
0  sw      0.199189           0.2960            0.1083  ZIRSNI
1  ws      0.184428           0.2948            0.0937  ZIRSNI
2  fd      0.289800           0.2898            0.2898  ZIRSNI


In [4]:
dataframe = parse_keyboard_log("logs/HEN/SNI/key.log")
keyboard_df = extract_predefined_patterns(dataframe, 0, 10000)
print(keyboard_df)

  key  avg_duration  longestDuration  shortestDuration   class
0  ws      0.143767           0.2920            0.0618  HENSNI
1  sw      0.174726           0.2950            0.0700  HENSNI
2  w2      0.109270           0.1654            0.0614  HENSNI
3  2w      0.178333           0.2938            0.0857  HENSNI
4  df      0.050600           0.0506            0.0506  HENSNI
